In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import cv2
import keras
from keras.utils import to_categorical
from sklearn.metrics import accuracy_score


In [ ]:
def main():
    # Defining class names
    classes = define_classNames()
    
    # Reading csv file
    train_csv,test_csv = read_file()
    
    # Defining labels for training and testing sets
    y_train,y_test = define_labels(train_csv,test_csv)

    # Creating training & testing dataframes consisting columns "Path" and "Labels" only
    train_df,test_df = create_df(train_csv,test_csv,y_train,y_test)
    
    # Converting labels into categorical form
    y_train,y_test = categorize_labels(train_df,test_df)
    
#     # Reviewing random signs from training dataset
#     review_samples(train_df,classes)
    
    # Preprocessing training data
    training_data = preprocess_trainingData(train_df)
    
    # Preprocessing testing data
    testing_data = preprocess_testingData(test_df)
    
    # Creating model
    model = create_model()
    print(model)
    
    # Compile model
    model = compile_model(model)
    
    # Defining keras callback
    class myCallback(keras.callbacks.Callback):
      def on_epoch_end(self,epoch,logs={}):
        if(logs.get('accuracy')> 0.99):
          print("\nReached target accuracy so cancelling training!")
          self.model.stop_training = True
    callback = myCallback()
    
    # Train model with training data
    model = train_model(model,training_data,y_train,callback)
    
    # Predicting test set
    pred = predictTestSet(model,testing_data)
    
    # Predicting first 10 images using our trained model
    predict_samples(model,testing_data,test_df,classes)
    
    # Finding Accuracy score
    find_accuracy(test_df,pred)
    
    
def define_classNames():
    classes = { 0: "Speed limit (20km/h)", 
               1: "Speed limit (30km/h)", 
               2: "Speed limit (50km/h)", 
               3: "Speed limit (60km/h)", 
               4: "Speed limit (70km/h)", 
               5: "Speed limit (80km/h)",
               6: "End of Speed limit (80km/h)", 
               7: "Speed limit (100km/h)", 
               8: "Speed limit (120km/h)",
               9: "No passing", 
               10: "No passing veh over 3.5 tons", 
               11: "Right-of-way at intersection",
               12: "Priority road", 
               13: "Yield", 
               14: "Stop", 
               15: "No vehicles", 
               16: "Veh > 3.5 tons prohibited",
               17: "No entry", 
               18: "General caution", 
               19: "Dangerous curve left", 
               20: "Dangerous curve right", 
               21: "Double curve", 
               22: "Bumpy road", 
               23: "Slippery road", 
               24: "Road narrows on the right",
               25: "Road work", 
               26: "Traffic signals", 
               27: "Pedestrians", 
               28: "Children crossing", 
               29: "Bicycle crossing",
               30: "Beware of ice/snow", 
               31: "Wild animals crossing", 
               32: "End speed + passing limits", 
               33: "Turn right ahead", 
               34: "Turn left ahead", 
               35: "Ahead only", 
               36: "Go straight or right", 
               37: "Go straight or left", 
               38: "Keep right", 
               39: "Keep Left", 
               40: "Roundabout mandatory",
               41: "End of no passing", 
               42: "End of no passing veh > 3.5 tons"}
    return classes

def read_file():
    train_csv = pd.read_csv('/kaggle/input/gtsrb-german-traffic-sign/Train.csv')
    test_csv = pd.read_csv('/kaggle/input/gtsrb-german-traffic-sign/Test.csv')
    return train_csv,test_csv
    
def define_labels(train_csv,test_csv):
    y_train = list(train_csv['ClassId'])
    y_test = list(test_csv['ClassId'])
    return y_train,y_test
    
def create_df(train_csv,test_csv,y_train,y_test):
    training_images_path = list('/kaggle/input/gtsrb-german-traffic-sign/' + train_csv['Path'])
    testing_images_path = list('/kaggle/input/gtsrb-german-traffic-sign/' + test_csv['Path'])
    
    train_df = pd.DataFrame(data = {'Path':training_images_path,'Labels':y_train})
    test_df = pd.DataFrame(data = {'Path':testing_images_path,'Labels':y_test})
    return train_df,test_df
    
def categorize_labels(train_df,test_df):
    y_train = to_categorical(train_df.Labels)
    y_test = to_categorical(test_df.Labels)
    return y_train,y_test
    
# def review_samples(train_df,classes):
#     print("Reviewing samples from training set:")
#     plt.figure(figsize=(12,25))
#     images = np.random.randint(0,len(train_df),4)
#     j=1
#     for img in images:
#         plt.subplot(1,4,j)
#         eachImg = cv2.imread(train_df['Path'][img])
#         eachImg = cv2.cvtColor(eachImg,cv2.COLOR_BGR2GRAY)
#         eachImg = eachImg/255
#         plt.imshow(eachImg)
#         plt.axis('off')
#         plt.title('{}({})'.format(classes[(train_df['Labels'][img])] ,train_df['Labels'][img]))
#         j+=1
        
def preprocess_trainingData(train_df):
    training_data = []
    for num in range(len(train_df)):
        img = cv2.imread(train_df['Path'][num])
        img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        img = cv2.equalizeHist(img.astype('uint8'))
        img = img/255
        img = cv2.resize(img,(33,33))
        training_data.append(img)
        
    training_data = np.array(training_data)
    training_data = training_data.reshape(training_data.shape[0],training_data.shape[1],training_data.shape[2],1)
    return training_data
    
def preprocess_testingData(test_df):
    testing_data = []
    for num in range(len(test_df)):
        img = cv2.imread(test_df['Path'][num])
        img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        img = cv2.equalizeHist(img.astype('uint8'))
        img = img/255
        img = cv2.resize(img,(33,33))
        testing_data.append(img)
        
    testing_data = np.array(testing_data)
    testing_data = testing_data.reshape(testing_data.shape[0],testing_data.shape[1],testing_data.shape[2],1)
    return testing_data
    
def create_model():
    model = keras.models.Sequential([
         keras.layers.Conv2D(filters=32, kernel_size=(5,5), activation='relu', input_shape=[33,33,1]),
         keras.layers.MaxPooling2D(pool_size=(2, 2)),
         keras.layers.Flatten(input_shape=(33,33)),
         keras.layers.Dense(units=256, activation='relu'),
         keras.layers.Dropout(0.2),
         keras.layers.Dense(units=43, activation='softmax')
    ])
    return model

def compile_model(model):
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model
    
def train_model(model,training_data,y_train,callback):
    model.fit(training_data,y_train, epochs=30, callbacks=[callback])
    return model
    
def predictTestSet(model,testing_data):
    pred = model.predict(testing_data)
    pred = np.argmax(pred, axis = 1)
    return pred
    
def predict_samples(model,testing_data,test_df,classes):
    pred_sample=[]
    for i in range(0,10):
        pred_temp = model.predict(np.expand_dims(testing_data[i],axis=0)).round(2)
        pred_temp = np.argmax(pred_temp, axis = 1)
        pred_sample.append(pred_temp)
    
    # Visualizing our predicted samples
    print("Visualizing our predicted samples:")
    plt.figure(figsize=(20,9))
    j=1
    for img in range(0,10):
        plt.subplot(2,5,j)
        eachImg = cv2.imread(test_df['Path'][img])
        eachImg = cv2.cvtColor(eachImg,cv2.COLOR_BGR2GRAY)
        eachImg = eachImg/255
        plt.imshow(eachImg)
        plt.axis('off')
        plt.title('Actual:({}){} \nPredicted:({}){}'.format( test_df['Labels'][img],classes[test_df['Labels'][img]], pred_sample[img][0],classes[pred_sample[img][0]] ))
        j+=1
        
#     return pred_sample
        
def find_accuracy(test_df,pred):
    acc = accuracy_score(test_df['Labels'],pred)
    print("Accuracy={}%".format(acc.round(3)*100))
    
if __name__ == "__main__":
    main()